In [1]:
# coding=utf-8
import os
import re
import csv
import time
import xlrd
# 记录所有数据
data_all = {}
# 时长不足
data_no_time = []
# 没签到  
data_no_sign = []
# 保存头部
csv_headers = ['学号', '姓名', '观看次数', '签到次数']
csv_one_headers = ['学号','名字','观看直播时长','签到','直播时间']

In [2]:
class Utils(object):
    """
    工具类
    """
    def select_number(self, text):
        """
        清洗出数字
        :text:文本
        """
        number_tmp = re.search(r'\d+', text) # 避免格式错误导致NoneType
        number = '' # 学号
        if number_tmp:
            number = number_tmp.group()
        return number
    def select_no_number(self, text):
        """
        清除数字的文本
        :text:文本
        """
        txt = re.sub('[^A-Za-z\u4e00-\u9fa5]', '', text).lstrip()
        return txt
    def select_bool(self, text):
        """
        识别bool类型
        :text:文本
        """
        txt = re.search(r'是|否',text)
        if txt:
            txt = txt.group()
        else: # 老师没签到
            txt = '是'
        return txt
    def clean_number(self, text):
        """
        清洗学号
        :text:学号
        """
        num = len(text)
        if num == 3 or num == 11:
            return text
        else:
            return '格式错误'
    def clean_time(self, resource_text, dist_text):
        """
        清洗时长
        :text: 时长
        """
        txt = resource_text
        if not resource_text:
            return '不足一分钟'
#         if int(resource_text) > int(dist_text):
#             print(resource_text,dist_text,'================================================================')
#             txt = '错误时间'
        return txt

In [3]:
class Student(object):
    """
    数据处理类
    """
    def clean_one(self, excel):
        """
        处理单个excel
        :excel:文件路径
        """
        # 单个文本数据
        data_one_all = {}
        # 打开文件
        data = xlrd.open_workbook(excel)
        data_sheet = data.sheet_by_name("数据导出")
        all_row = data_sheet.nrows  # 总行数
        utils = Utils()
        all_time = utils.select_number(data_sheet.row_values(2)[1]) # 直播真实时长
        view_time =  ' '.join(data_sheet.row_values(2)[0].split(' ')[:2]) # 直播时间
        
        is_sign = False # 3月份没签到功能
        if len(data_sheet.row_values(5)) > 10:
            is_sign = True
        else:
            print('这节课还没签到功能：',view_time)
            
        for i in range(5, all_row):
            row_date = data_sheet.row_values(i)     
            number = utils.select_number(row_date[3]) # 学号
            number = utils.clean_number(number) # 清洗后的学号
            name = utils.select_no_number(row_date[3]) # 姓名
            time = utils.select_number(row_date[7]) # 直播时长
            time = utils.clean_time(time, all_time) # 清洗后的时长
            
            if is_sign:
                sign = utils.select_bool(row_date[10]) # 是否签到
            else:
                sign = '是'
                
            if data_one_all.setdefault(name,0):
                time_tmp = 0
                if data_one_all[name]['观看直播时长'] != '不足一分钟':
                    time_tmp += int(data_one_all[name]['观看直播时长'])    
                if time != '不足一分钟':
                    time_tmp += int(time)
                data_one_all[name]['观看直播时长'] = str(time_tmp)
                if sign == '是' and data_one_all[name]['签到'] == '否':
                    data_one_all[name]['签到'] = '是'
            else:
                data_one_all[name] = {
                    '学号': number,
                    '名字': name,
                    '观看直播时长': time,
                    '签到' : sign,
                    '直播时间': view_time
                }
        return data_one_all , all_time
    def stat_data(self, data, all_time):
        """
        统计所有excel文件
        :all_data:单个清洗后的数据
        :all_time:当次直播时长
        """
        for one_data in data:
            time_tmp = False
            if data[one_data]['观看直播时长'] != '不足一分钟':
                time_tmp = int(data[one_data]['观看直播时长']) > 60  # 控制时长筛选条件
            sign_tmp = data[one_data]['签到'] == '是'
            if not time_tmp:
                data_no_time.append(data[one_data])
            if not sign_tmp:
                data_no_sign.append(data[one_data]) 
            if data_all.setdefault(one_data, 0):
                data_all[data[one_data]['名字']]['观看次数'] += time_tmp # 观看 + 1 
                data_all[data[one_data]['名字']]['签到次数'] += sign_tmp # 签到 + 1
            else:
                """
                "姓名" ：{
                    "姓名" ： "xxx",
                    "学号" : "xxx",
                    "签到次数": "xxx",
                    "直播总次数": "xx",
                    "观看视频次数": ""
                }
                """
                data_all[data[one_data]['名字']] = {
                    '学号': data[one_data]['学号'],
                    '姓名': data[one_data]['名字'],
                    '观看次数': int(time_tmp),
                    '签到次数': int(sign_tmp)
                }
    def clean_error(self,data):
        """
        清洗学号格式错误人员
        :data:清洗的数据
        @return num 总人数
        """
        num = 0
        for data_tmp in list(data):
            num += 1
            if isinstance(data,dict):
                if data[data_tmp]['学号'] == '格式错误':
                    print(data[data_tmp])
                    num -= 1
                    data.pop(data_tmp)
            else:
                if data_tmp['学号'] == '格式错误':
                    num -= 1
                    data.remove(data_tmp)
        return num

In [4]:
class Save(object):
    
    """
    数据存储类
    """
    def csv_save(self, data, csv_name, headers):
        """
        csv存储数据
        :data:存储的数据
        :文件名字
        """
        utils = Utils()
        if isinstance(data,dict): # dict转list
            data = [i for i in data.values()]
        data = sorted(data, key = lambda x : x["学号"]) # 排序
        with open(csv_name, mode = 'a', newline='') as f:
            writer = csv.DictWriter(f, headers)
            for row in data:
                writer.writerow(row)

In [5]:
# 每次更新,防止追加
data_all= {}
data_no_sign = []
data_no_time = []
if __name__ == '__main__':
    student = Student()
    # 写入全部数据头部     
    with open('data_all.csv',mode = 'a', newline='') as f:
            writer = csv.DictWriter(f, csv_headers)
            writer.writeheader()
    # 写入时长未到&没签到
    with open('data_error.csv',mode = 'a', newline='') as f:
            writer = csv.DictWriter(f, csv_one_headers)
            writer.writeheader()
    for path in os.listdir("data"):
        path = 'data/{}'.format(path)
        data , all_time = student.clean_one(path) # 单个excel文件数据ss
        student.stat_data(data,all_time) # 所有数据
    num = student.clean_error(data_all) # 清洗data_all学号格式错误的人员
    num_no_time = student.clean_error(data_no_time) # 清洗data_no_time学号格式错误的人员s
    num_no_sign = student.clean_error(data_no_sign) # 清洗data_no_sign学号格式错误的人员
    # 存储全部数据
    save = Save()
    save.csv_save(data_all, 'data_all.csv',  csv_headers)
    save.csv_save(data_no_time,'data_error.csv',csv_one_headers) # 时长未到
    save.csv_save(data_no_sign,'data_error.csv',csv_one_headers) # 未签到

这节课还没签到功能： 2020-03-11 09:50:31
这节课还没签到功能： 2020-03-18 09:42:12
这节课还没签到功能： 2020-03-19 07:41:54
{'学号': '格式错误', '姓名': '崔宗楠', '观看次数': 19, '签到次数': 16}
{'学号': '格式错误', '姓名': 'Mambaout', '观看次数': 0, '签到次数': 2}
{'学号': '格式错误', '姓名': '荒于嬉', '观看次数': 0, '签到次数': 1}
{'学号': '格式错误', '姓名': '张连彬', '观看次数': 21, '签到次数': 14}
{'学号': '格式错误', '姓名': '瞒心', '观看次数': 0, '签到次数': 1}
{'学号': '格式错误', '姓名': '年年', '观看次数': 0, '签到次数': 3}
{'学号': '格式错误', '姓名': '躯壳', '观看次数': 0, '签到次数': 1}
{'学号': '格式错误', '姓名': '信息侯泽军', '观看次数': 0, '签到次数': 1}
{'学号': '格式错误', '姓名': '黄皆丽', '观看次数': 0, '签到次数': 1}
{'学号': '格式错误', '姓名': '信息梁旭', '观看次数': 0, '签到次数': 1}
{'学号': '格式错误', '姓名': '陈婉婷', '观看次数': 0, '签到次数': 1}
{'学号': '格式错误', '姓名': '韦绍军', '观看次数': 0, '签到次数': 1}
{'学号': '格式错误', '姓名': '冯敏玲', '观看次数': 0, '签到次数': 1}
{'学号': '格式错误', '姓名': '张幸霖', '观看次数': 0, '签到次数': 1}
{'学号': '格式错误', '姓名': '景星杰', '观看次数': 0, '签到次数': 1}
{'学号': '格式错误', '姓名': '信息戴震泓', '观看次数': 0, '签到次数': 1}
{'学号': '格式错误', '姓名': '梦半醒', '观看次数': 0, '签到次数': 1}
{'学号': '格式错误', '姓名': '信息傅晓琳', '观看次数': 0, '签到次数